In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cv2 
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [3]:
base_train_dir = "../input/lung-disease-classification/train_/train_"
base_test_dir = "../input/lung-disease-classification/test_/test_"

In [4]:
image_length_size = 300
image_height_size = 200
batch_size = 32

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip  = True,
    fill_mode='nearest',
    validation_split = 0.1
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    base_train_dir,
    target_size = (image_length_size,image_height_size),
    batch_size = batch_size,
    class_mode='sparse',
    subset = 'training',
    shuffle = True
)

Found 10922 images belonging to 5 classes.


In [7]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip =True,
    fill_mode = 'nearest',
    validation_split = 0.1
)

In [8]:
test_generator = test_datagen.flow_from_directory(
    base_test_dir,
    target_size = (image_length_size,image_height_size),
    batch_size = batch_size,
    class_mode='sparse',
    subset = 'validation',
    shuffle=True
)

Found 137 images belonging to 5 classes.


VGG 16 MODEL 

In [9]:
vgg_model = VGG16(include_top=False, #top 3 fully connected layers are discarded
                  weights='imagenet',
                  input_shape=(200,300,3),
                  pooling='avg')

2023-01-27 19:59:31.407251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 19:59:31.496637: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 19:59:31.497559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 19:59:31.499462: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 2s 0us/step


In [10]:
vgg_model.trainable = False

In [11]:
import tensorflow as tf

In [12]:
model = tf.keras.models.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8

In [14]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [15]:
history = model.fit(
            train_generator,
            epochs=30,
            steps_per_epoch = 300,
    # 92k images = batch_size*steps
    #i.e steps = 10k/32 = 312
            #steps_per_epoch=total_train_batches//hvd.size(),
            validation_data=test_generator,
            verbose=1
            )

2023-01-27 19:59:38.225670: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2023-01-27 19:59:40.653623: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


300/300 [==============================] - 213s 676ms/step - loss: 1.3253 - accuracy: 0.4682 - val_loss: 1.2135 - val_accuracy: 0.4964
Epoch 2/30
300/300 [==============================] - 174s 578ms/step - loss: 1.2286 - accuracy: 0.5056 - val_loss: 1.0827 - val_accuracy: 0.5401
Epoch 3/30
300/300 [==============================] - 162s 541ms/step - loss: 1.1062 - accuracy: 0.5656 - val_loss: 0.9963 - val_accuracy: 0.6131
Epoch 4/30
300/300 [==============================] - 169s 564ms/step - loss: 1.0004 - accuracy: 0.6276 - val_loss: 0.9843 - val_accuracy: 0.6569
Epoch 5/30
300/300 [==============================] - 167s 557ms/step - loss: 0.9022 - accuracy: 0.6715 - val_loss: 0.9466 - val_accuracy: 0.6423
Epoch 6/30
300/300 [==============================] - 162s 540ms/step - loss: 0.8431 - accuracy: 0.6812 - val_loss: 0.9501 - val_accuracy: 0.6131
Epoch 7/30
300/300 [==============================] - 164s 545ms/step - loss: 0.8034 - accuracy: 0.6975 - val_loss: 0.8508 - val_accura

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [17]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [18]:
vgg_19 = VGG19(include_top=False, #top 3 fully connected layers are discarded
                  weights='imagenet',
                  input_shape=(200,300,3),
                  pooling='avg')

80150528/80134624 [==============================] - 3s 0us/step


In [19]:
vgg_19.trainable = False

In [20]:
model2 = tf.keras.models.Sequential()
model2.add(vgg_19)
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(256,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(128,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(64,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(32,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(16,activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(5,activation='softmax'))

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 512)               20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)               

In [22]:
model2.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [23]:
history2 = model2.fit(
            train_generator,
            epochs=30,
            steps_per_epoch = 300,
    # 92k images = batch_size*steps
    #i.e steps = 10k/32 = 312
            #steps_per_epoch=total_train_batches//hvd.size(),
            validation_data=test_generator,
            verbose=1
            )

Epoch 1/30
300/300 [==============================] - 165s 545ms/step - loss: 1.4764 - accuracy: 0.3628 - val_loss: 1.3120 - val_accuracy: 0.4964
Epoch 2/30
300/300 [==============================] - 166s 554ms/step - loss: 1.3412 - accuracy: 0.4677 - val_loss: 1.2412 - val_accuracy: 0.4964
Epoch 3/30
300/300 [==============================] - 165s 551ms/step - loss: 1.2838 - accuracy: 0.4973 - val_loss: 1.1604 - val_accuracy: 0.5255
Epoch 4/30
300/300 [==============================] - 163s 542ms/step - loss: 1.1913 - accuracy: 0.5593 - val_loss: 1.0627 - val_accuracy: 0.6204
Epoch 5/30
300/300 [==============================] - 168s 559ms/step - loss: 1.1058 - accuracy: 0.6175 - val_loss: 1.0598 - val_accuracy: 0.5839
Epoch 6/30
300/300 [==============================] - 162s 540ms/step - loss: 1.0218 - accuracy: 0.6554 - val_loss: 0.9919 - val_accuracy: 0.6131
Epoch 7/30
300/300 [==============================] - 164s 547ms/step - loss: 0.9692 - accuracy: 0.6683 - val_loss: 0.9723 -

In [24]:
# serialize model to JSON
model2_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights("model2.h5")

In [25]:
# serialize model to JSON
model2_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights("model2.h5")